In [1]:
import json
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/apollocate/4300-Final-Project-2023/

/content/drive/MyDrive/apollocate/4300-Final-Project-2023


In [2]:
f = open('backend/dataset/wiki_scraping/wiki_texts.json')
wiki_texts = json.load(f)

In [3]:
big_df = pd.read_csv('backend/dataset/big_df_edited.csv')

In [4]:
big_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
big_df['tknzd_lyrics'] = big_df['tknzd_lyrics'].apply(ast.literal_eval)
big_df['emotions'] = big_df['emotions'].apply(ast.literal_eval)
big_df['social_tags'] = big_df['social_tags'].apply(ast.literal_eval)

In [8]:
big_df.shape

(34462, 14)

In [9]:
drop_rows = []
for row in big_df.iterrows():
    if row[1]['emotions'] == []:
        drop_rows.append(row[0])
big_df.drop(drop_rows, inplace=True)

In [10]:
big_df.shape

(34462, 14)

In [12]:
# my_stop_words = text.ENGLISH_STOP_WORDS.union({'city'})
vectorizer = TfidfVectorizer(max_df = 0.8, min_df = 10, norm='l2')#, stop_words = list(my_stop_words))
X = vectorizer.fit_transform(big_df['tknzd_lyrics'].apply(lambda x: " ".join(x)))

In [13]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [14]:
index_to_word = {i:v for i, v in enumerate(vectorizer.get_feature_names_out())}
word_to_index = {v:i for i, v in enumerate(vectorizer.get_feature_names_out())}

In [15]:
with open('word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f)
with open('index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f)

In [16]:
with open('song_tf_idf.pickle', 'wb') as f:
    pickle.dump(X, f)

In [17]:
song_to_index = {s:i for i, s in enumerate(big_df['title'])}
index_to_song = {i:s for i, s in enumerate(big_df['title'])}

In [18]:
with open('song_to_index.pkl', 'wb') as f:
    pickle.dump(song_to_index, f)
with open('index_to_song.pkl', 'wb') as f:
    pickle.dump(index_to_song, f)

# wiki

In [19]:
from dataset import wiki_scraping

In [20]:
print(len(wiki_texts['Tel Aviv']))

11191


In [21]:
wiki_corpus = []
for ls in list(wiki_texts.values()):
    wiki_corpus.append(" ".join(ls))

In [22]:
vec2 = vectorizer.transform(wiki_corpus)
vec2 = vec2.toarray()
X = X.toarray()

In [23]:
vec2.sum(axis=1)

array([13.23732851, 18.6994788 ,  8.71294141, 10.52818644, 10.85053303,
       11.61173687, 11.29440799,  3.72468532, 11.93210564, 13.42178719,
       12.04420778, 11.08377508,  3.14946011, 19.50359818, 11.23738959,
       11.13433235, 10.97519174, 10.94497972,  2.65657314, 10.84608893,
       10.55599349, 10.68164386,  3.25946005, 12.46706059, 10.68392845,
       10.47639   , 10.62528014, 16.56103246, 14.5215441 , 12.79915179,
        9.56817781, 15.86057865, 17.2775259 , 13.80991462, 16.31199392,
       16.42247152, 17.07313671, 17.63321961, 14.79179067, 18.33283399,
       17.54404677, 18.03810555, 11.96791668, 10.87464697, 11.94101457,
       19.0396642 , 11.37038882])

In [24]:
print(vec2.shape)
print(X.shape)

(47, 13999)
(34462, 13999)


In [25]:
vec2[2,:]

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
loc_to_index = {cty:i for i, cty in enumerate(wiki_texts.keys())}

In [27]:
def cos_sim(city, song):
    city_i = loc_to_index[city]
    song_i = song_to_index[song]
    city_vec = vec2[city_i, :]
    song_vec = X[song_i, :]
    denom = np.linalg.norm(city_vec) * np.linalg.norm(song_vec)
    num = city_vec @ song_vec
    return (num ) /  (denom )

In [28]:
vec2[2,:].shape

(13999,)

In [29]:
def best_songs_for_city(city):
    best = []
    for song in song_to_index:
        sim = cos_sim(city, song)
        best.append((song, sim))
    srtd = sorted(best, key = lambda x: x[1], reverse=True)
    for t in srtd[:10]:
        print("Song: ", t[0], "  Score: {:.3f}".format(t[1]))

In [30]:
best_songs_for_city("New York City")

<ipython-input-27-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  I Love NYC   Score: 0.681
Song:  New York   Score: 0.608
Song:  New York City Boy   Score: 0.605
Song:  New York City Cops   Score: 0.551
Song:  New York Minute   Score: 0.525
Song:  King Of New York   Score: 0.448
Song:  The Incumbent   Score: 0.344
Song:  Sex City   Score: 0.332
Song:  City Of Love   Score: 0.327
Song:  New York Slave   Score: 0.306


In [33]:
best_songs_for_city("London")

<ipython-input-29-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  London Bridge   Score: 0.762
Song:  London Is The Place For Me   Score: 0.727
Song:  London Calling   Score: 0.618
Song:  Glamorous Glue   Score: 0.523
Song:  London Loves   Score: 0.514
Song:  Street Fighting Man   Score: 0.281
Song:  Your Embrace   Score: 0.225
Song:  The Vanishing   Score: 0.185
Song:  Round Here   Score: 0.150
Song:  Delaney   Score: 0.143


In [34]:
best_songs_for_city("Tel Aviv")

<ipython-input-29-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  Victime de la mode   Score: 0.259
Song:  Redrum   Score: 0.209
Song:  Red Velvet   Score: 0.126
Song:  The Ballot or the Bullet   Score: 0.055
Song:  Hitten   Score: 0.040
Song:  100%   Score: 0.039
Song:  Oh Jerusalem   Score: 0.037
Song:  Poster Princess   Score: 0.037
Song:  Home Life   Score: 0.035
Song:  Intro   Score: 0.035


In [35]:
best_songs_for_city("Tokyo")

<ipython-input-29-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  Tokyo Witch   Score: 0.280
Song:  Panda Bear   Score: 0.163
Song:  Never Ending Summer   Score: 0.142
Song:  Award Tour   Score: 0.104
Song:  Back 4 U   Score: 0.075
Song:  Bodhisattva   Score: 0.072
Song:  Impossible Germany   Score: 0.063
Song:  Harajuku Girls   Score: 0.059
Song:  Da Joint   Score: 0.055
Song:  Losing My Edge   Score: 0.050


In [36]:
best_songs_for_city("Mumbai")

<ipython-input-29-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  Indian Girl   Score: 0.158
Song:  Citysong   Score: 0.129
Song:  The Ballot or the Bullet   Score: 0.124
Song:  Fireworks   Score: 0.112
Song:  30 Century Man   Score: 0.107
Song:  21st Century Digital Boy   Score: 0.088
Song:  Twentieth Century Fox   Score: 0.084
Song:  Suburban Home   Score: 0.083
Song:  The Kids   Score: 0.082
Song:  Flux   Score: 0.080


In [31]:
best_songs_for_city("Seoul")

<ipython-input-27-081fac839062>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (num ) /  (denom )


Song:  City of Night   Score: 0.263
Song:  City Of Love   Score: 0.263
Song:  Sex City   Score: 0.250
Song:  Thin Blue Flame   Score: 0.237
Song:  1% Of One   Score: 0.222
Song:  City of Devils   Score: 0.221
Song:  The Best of Times   Score: 0.221
Song:  We Built This City   Score: 0.219
Song:  The Buzz Kill   Score: 0.219
Song:  An Erotic Alchemy   Score: 0.217


In [32]:
with open('wiki_tf_idf.pkl', 'wb') as f:
    pickle.dump(vec2, f)

In [39]:
big_df.loc[big_df['title']=='DVNO']

,title,tag,artist,year,views,features,lyrics,id,song_id,emotions,social_tags,tknzd_lyrics
1779,DVNO,rock,Justice,2007,30010,"{""Mehdi Pinson""}",[Verse 1]\nIt's always the same\nAlways ashame...,100497,TRNPKRK128F429831C,"[(party, 13), (energetic, 1), (fun, 1), (sexy,...","[(electronic, 100), (dance, 67), (electro, 61)...","[its, always, the, same, always, ashamed, stor..."


In [2]:
vectorizer

NameError: name 'vectorizer' is not defined

In [33]:
with open('loc_to_index.pkl', 'wb') as f:
    pickle.dump(loc_to_index, f)

In [34]:
loc_to_index['Tel Aviv']

25